In [1]:
import sys
!{sys.executable} -m pip install textblob
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install tensorflow

In [2]:
import pandas as pd
import nltk
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec, KeyedVectors
from keras.models import Sequential
from keras.layers import Dense, Dropout
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from keras.callbacks import EarlyStopping
import sqlite3

# Load Data
conn = sqlite3.connect('database.sqlite')
query = "SELECT * FROM May2015 LIMIT 500000;"
df = pd.read_sql(query, conn)
conn.close()

# Preprocessing
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    words = nltk.word_tokenize(text.lower())
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words and word.isalpha()]
    return ' '.join(words)

df = df.dropna(subset=['body'])
df['body'] = df['body'].apply(preprocess_text)
df = df[df['body'].str.len() > 10]

# Sentiment Analysis
df['polarity'] = df['body'].apply(lambda x: TextBlob(x).sentiment.polarity)
df['subjectivity'] = df['body'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

# Word2Vec Embeddings
tfidf = TfidfVectorizer()
tfidf.fit(df['body'])
tfidf_dict = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

def get_average_word2vec(tokens_list, vector, tfidf_weights, k=300):
    if len(tokens_list) < 1:
        return np.zeros(k)
    vectorized = [vector[word] * tfidf_weights.get(word, 1) if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

model_path = "GoogleNews-vectors-negative300.bin"
word2vec_model = KeyedVectors.load_word2vec_format(model_path, binary=True)
df['word2vec'] = df['body'].apply(lambda x: get_average_word2vec(x.split(), word2vec_model, tfidf_dict))

# Preparing Features
X = pd.concat([df['polarity'], df['subjectivity'], pd.DataFrame(df['word2vec'].tolist())], axis=1)
X.columns = X.columns.astype(str)  # Convert column names to strings
y = df['score'].apply(lambda x: 1 if x > 0 else 0).values

X = X.dropna()
y = y[X.index]





# Handling Class Imbalance using SMOTE
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

# Logistic Regression Model as a baseline
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report
print("Logistic Regression Results:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Neural Network Model
model = Sequential()
model.add(Dense(units=128, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

y_pred_nn = model.predict(X_test)
y_pred_nn = [1 if p >= 0.5 else 0 for p in y_pred_nn]

print("\nNeural Network Results:")
print("Accuracy:", accuracy_score(y_test, y_pred_nn))
print(classification_report(y_test, y_pred_nn))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sanks04/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/sanks04/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Logistic Regression Results:
Accuracy: 0.6287303103008322
              precision    recall  f1-score   support

           0       0.63      0.65      0.64     62521
           1       0.63      0.61      0.62     62100

    accuracy                           0.63    124621
   macro avg       0.63      0.63      0.63    124621
weighted avg       0.63      0.63      0.63    124621

Epoch 1/50
15578/15578 [==============================] - 10s 630us/step - loss: 0.6025 - accuracy: 0.6782 - val_loss: 0.5796 - val_accuracy: 0.6876
Epoch 2/50
15578/15578 [==============================] - 10s 630us/step - loss: 0.5734 - accuracy: 0.7021 - val_loss: 0.5493 - val_accuracy: 0.7109
Epoch 3/50
15578/15578 [==============================] - 10s 640us/step - loss: 0.5584 - accuracy: 0.7128 - val_loss: 0.5300 - val_accuracy: 0.7269
Epoch 4/50
15578/15578 [==============================] - 10s 622us/step - loss: 0.5480 - accuracy: 0.7220 - val_loss: 0.5276 - val_accuracy: 0.7332
Epoch 5/50
15578/15